<a href="https://colab.research.google.com/drive/1aGZv3bNDvrEOOCq78lSYREGiO7jpTgt7?usp=sharing" target="_parent">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

#### [Huggingface Models](https://huggingface.co/)

Hugging Face LLMs need a T4 GPU. In Colab, go to the top-right drop down menu → Change runtime type → select T4 GPU.

### Install required libraries

In [3]:
# langchain-huggingface: LangChain integration with HuggingFace models
# text-generation: Inference client for HuggingFace text-generation models
# transformers: Core HuggingFace library for pretrained models (LLMs, embeddings, etc.)
# langchainhub: Access pre-built LangChain prompts, chains, and tools from the Hub
# bitsandbytes: Lightweight quantization + efficient GPU memory usage (4-bit/8-bit LLMs)
# accelerate: HuggingFace utility to optimize training/inference across devices
!pip install --upgrade --quiet \
    langchain-huggingface \
    text-generation \
    transformers \
    langchainhub \
    bitsandbytes \
    accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 130.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.5 MB/s eta 0:00:00


### Import related libraries

In [4]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import os
import getpass

### 🔑 Provide Huggingface API Key

[Huggingface API Generation Key Link](https://huggingface.co/settings/tokens)


In [5]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass()

··········


### Code Explanation:

This code sets up **4-bit quantization** using Hugging Face’s `bitsandbytes` library. Quantization makes large language models run faster and use less GPU memory by storing weights in lower precision.

1. `load_in_4bit`=True → Loads the model in 4-bit precision instead of full 16/32-bit.

2. `bnb_4bit_quant_type`="nf4" → Uses **NF4 (Normalized Float 4)**, a quantization method that gives better accuracy.

3. `bnb_4bit_compute_dtype`="float16" → Performs computations in 16-bit floating-point for balance between speed and precision.

4. `bnb_4bit_use_double_quant`=True → Applies an extra quantization step to save even more memory.

👉 In short: this config lets you run **big LLMs on smaller GPUs** (like Colab T4) without running out of memory.

In [6]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

### Code Explanation:

This code loads a Hugging Face LLM and wraps it with LangChain so you can use it like a chat model.

1. `HuggingFacePipeline.from_model_id(...)` → Loads the model from Hugging Face Hub by its ID (deepseek-ai/DeepSeek-R1-Distill-Llama-8B) and sets it up as a text-generation pipeline.

    - `task="text-generation"` → Defines the model’s job (generate text).

    - pipeline_kwargs:

        - `max_new_tokens`=512 → Limits the output length.

        - `do_sample`=False → Disables randomness (makes output more deterministic).

        - `repetition_penalty`=1.03 → Prevents the model from repeating the same text.

        - `return_full_text`=False → Returns only the new response (not the prompt + response).

    - `model_kwargs`={"quantization_config": quantization_config} → Applies the 4-bit quantization settings from earlier to save GPU memory.

2. `llm` → The raw Hugging Face model pipeline.

3. `chat_model` = ChatHuggingFace(llm=llm) → Wraps the pipeline into a **LangChain-compatible** chat model, so you can easily build chatbots or RAG workflows with it.

👉 In short: this sets up **DeepSeek Llama-8B (quantized)** as a chat-ready model in LangChain.

In [7]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
        return_full_text=False,
    ),
    model_kwargs={"quantization_config": quantization_config},
)

chat_model = ChatHuggingFace(llm=llm)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Code Explanation

This code shows how to **send messages to the chat model** and get a response, similar to a conversation.

1. `SystemMessage(content="...")` → Sets the **role** or instructions for the AI (e.g., telling it to act like a helpful assistant).

2. `HumanMessage(content="...")` → Represents the **user’s input** or question.

3. `messages = [...]` → Collects both system and human messages into a list, which is passed to the model.

4. `chat_model.invoke(messages)` → Sends the conversation to the model and returns an AI response.

5. `print(ai_msg.content)` → Prints the model’s reply text.

👉 In short: this creates a **chat-style interaction** where you give the AI context (system role) + a user query, and the model responds just like ChatGPT.

In [8]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(
        content="What happens when an unstoppable force meets an immovable object?"
    ),
]

ai_msg = chat_model.invoke(messages)
print(ai_msg.content)

Okay, so I'm trying to figure out what happens when an unstoppable force meets an immovable object. Hmm, this is an interesting question. Let me break it down.

First, I need to understand what each term means. An "unstoppable force" is something that can't be stopped or prevented, right? Like a natural disaster or a powerful entity with immense power. On the other hand, an "immovable object" is something that can't be moved or changed, like a mountain or a very strong structure.

So, if I imagine these two concepts colliding, what would happen? Well, if the force is unstoppable, it's going to try to move or change the object. But the object is immovable, meaning it won't budge. So, the force might try to push, pull, or break it, but the object remains in place.

I wonder if the force could eventually overcome the object. Maybe if the force is strong enough or persistent enough, it could cause some damage or even destroy the object. But then again, the object is immovable, so maybe it'